# Import library and read the excel file


In [53]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os

directory = "Incentive_CAL"
file_name = "COM incentive_Dec 2022_Duy_Anh.xlsx"
path = os.path.join(directory, file_name)
df = pd.read_excel(path, "Detail")
df = df.iloc[9:, 1:]

new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

#take rows that contain specific 'Onboarded' in 'Store status'
df = df[df["Store status"] == "Onboarded"]

In [54]:
df['KAM/KAE (Full name)'] = df['KAM/KAE (Full name)'].str.title()
df['CD (Full name)'] = df['CD (Full name)'].str.title()
df['GBM/Team lead (Full name)'] = df['GBM/Team lead (Full name)'].str.title()

In [55]:
KAMs = df["KAM/KAE (Full name)"].unique()

# Take list of GBM
CDs = df["CD (Full name)"].unique()
GBM_under = []
for CD in CDs:
    CD_df = df.loc[df["CD (Full name)"] == CD]
    GBM_under.append(list(CD_df["GBM/Team lead (Full name)"].unique()))

In [26]:
df.columns

Index(['Group brand', ' Brand ', 'Channel', 'Platform', 'Store',
       'Store status', 'CD (Full name)', 'CD (Email)', 'CD (Employee code)',
       'GBM/Team lead (Full name)', 'GBM/Team lead (Email)',
       'GBM/Team lead (Employee code)', 'KAM/KAE (Full name)',
       'KAM/KAE (Email)', 'KAM/KAE (Employee code)', 'Business mode',
       'B2B or B2C', 'Model for incentive', '%GP (on NMV)', '%GP (on PP)',
       '%GP (on RSP)', 'Rebate (Yes/No)', '% Promotion',
       '%GP (not covering CS outsource)', 'Join sub (Yes/No)', 'Subsidy spent',
       '% Subsidy', 'Target NMV BP', 'Target GP BP (simulated)',
       'Target NMV after', 'Target GP after (Simulated)', 'NMV actual',
       'GP actual (Simulated)', 'SLOB performance', 'SLOB Before',
       'SLOB After', 'SLOB', '% NMV', '% GP'],
      dtype='object', name=9)

# Sheet Specific

## Define all function for GBM cal specific

### Func to take GBM and KAM list

In [56]:
# Take list of GBM
GBMs = df["GBM/Team lead (Full name)"].unique()
KAM_under = []
for GBM in GBMs:
    GBM_df = df.loc[df["GBM/Team lead (Full name)"] == GBM]
    KAM_under.append(list(GBM_df["KAM/KAE (Full name)"].unique()))    

In [57]:
###### Func for sum of SLOB per groupbrand
def cal_sum_slob(df, col: str):
    gb = df["Group brand"].to_list()
    slob = df[col].to_list()
    s = 0
    hold = ""
    for i in range(len(gb)):
        if hold != gb[i]:
            s += slob[i]
            hold = gb[i]
        else:
            continue
    return s

# Convert df col to percentage
def int_to_per(df, col):
    df[col] = df[col] * 100
    df[col] = df[col].astype(str)
    df[col] = df[col].apply(lambda x: x[:5] + "%" if x != "" else "")
    return df[col]

# Write the func for creating df for each KAM under GBM
def take_KAM_df(KAM_name: str = None, GBM_name: str = None, CD_name: str = None):
    KAM_df = df
    if KAM_name != None:
        KAM_df = df[df["KAM/KAE (Full name)"] == KAM_name]
    if GBM_name != None:
        KAM_df = KAM_df[KAM_df["GBM/Team lead (Full name)"] == GBM_name]
    if CD_name != None:
        KAM_df = KAM_df[KAM_df["CD (Full name)"] == CD_name]
    cols = ["KAM/KAE (Full name)", "Group brand", "Store", "B2B or B2C", "NMV actual",
            'Target NMV after', "% NMV",'GP actual (Simulated)', 'Target GP after (Simulated)',
            "% GP", "SLOB performance", 'SLOB Before', 'SLOB After', 'SLOB']
    KAM_df = KAM_df[cols]
    return KAM_df

# Function Extract col to B2B and B2C
def extract_b_or_c(df):
    b_or_c_col = df["B2B or B2C"].to_list()
    
    nmv_actual = df["NMV actual"].to_list()
    nmv_target = df["Target NMV after"].to_list()
    gp_actual = df["GP actual (Simulated)"].to_list()
    gp_target = df["Target GP after (Simulated)"].to_list()
    
    per_nmv_col = df["% NMV"].to_list()
    per_gp_col = df["% GP"].to_list()
    
    no_row = len(b_or_c_col)
    
    nmv_actual_b2b = []
    nmv_target_b2b = []
    gp_actual_b2b = []
    gp_target_b2b = []
    
    nmv_actual_b2c = []
    nmv_target_b2c = []
    gp_actual_b2c = []
    gp_target_b2c = []
    
    per_nmv_b2b = []
    per_nmv_b2c = []
    per_gp_b2b = []
    per_gp_b2c = []

    for i in range(no_row):        
        if b_or_c_col[i] == "B2B":
            nmv_actual_b2b.append(nmv_actual[i])
            nmv_target_b2b.append(nmv_target[i])
            gp_actual_b2b.append(gp_actual[i])
            gp_target_b2b.append(gp_target[i])
            
            nmv_actual_b2c.append(0)
            nmv_target_b2c.append(0)
            gp_actual_b2c.append(0)
            gp_target_b2c.append(0)
            
            per_nmv_b2b.append(per_nmv_col[i])
            per_gp_b2b.append(per_gp_col[i])
            per_nmv_b2c.append(0)
            per_gp_b2c.append(0)
        elif b_or_c_col[i] == "B2C":
            nmv_actual_b2c.append(nmv_actual[i])
            nmv_target_b2c.append(nmv_target[i])
            gp_actual_b2c.append(gp_actual[i])
            gp_target_b2c.append(gp_target[i])
            
            nmv_actual_b2b.append(0)
            nmv_target_b2b.append(0)
            gp_actual_b2b.append(0)
            gp_target_b2b.append(0)
            
            per_nmv_b2c.append(per_nmv_col[i])
            per_gp_b2c.append(per_gp_col[i])
            per_nmv_b2b.append(0)
            per_gp_b2b.append(0)
    
    # drop % nmv, % gp and add % nmv and gp by b2b and b2c
    df = df.drop(['% NMV', '% GP', 'NMV actual', "Target NMV after", "GP actual (Simulated)", "Target GP after (Simulated)"], axis=1)
    
    df.insert(4, 'NMV actual B2B', nmv_actual_b2b)
    df.insert(5, 'NMV target B2B', nmv_target_b2b)
    df.insert(6, '% NMV B2B', per_nmv_b2b)
    
    df.insert(7, 'NMV actual B2C', nmv_actual_b2c)
    df.insert(8, 'NMV target B2C', nmv_target_b2c)
    df.insert(9, '% NMV B2C', per_nmv_b2c)

    df.insert(10, 'GP actual B2B', gp_actual_b2b)
    df.insert(11, 'GP target B2B', gp_target_b2b)
    df.insert(12, '% GP B2B', per_gp_b2b)
    
    df.insert(13, 'GP actual B2C', gp_actual_b2c)
    df.insert(14, 'GP target B2C', gp_target_b2c)
    df.insert(15, '% GP B2C', per_gp_b2c)
    return df

# Func for highlight_last_row
def highlight_last_row(df):
    color = 'white'
    if df.values[3] == 'KAM':
        color = 'yellow'
    if df.values[0] in GBMs:
        color = 'red'
    if df.values[0] in CDs:
        color = 'orange'

    return ['background-color: %s' % color]*len(df.values)

### Func for create df for 1 KAM

In [58]:
def take_detail_KAM(KAM_name: str = None, GBM_name: str = None, CD_name:str = None):
    KAM_df = take_KAM_df(KAM_name, GBM_name, CD_name)
    # extract b2b and b2c
    KAM_df = extract_b_or_c(KAM_df)
    
    # Add Total row
    no_group_brand = len(KAM_df["Group brand"].unique())
    no_store = len(KAM_df["Store"].unique())
    
    sum_nmv_actual_b2b = KAM_df["NMV actual B2B"].sum()
    sum_nmv_actual_b2c = KAM_df["NMV actual B2C"].sum()
    sum_nmv_target_b2b = KAM_df["NMV target B2B"].sum()
    sum_nmv_target_b2c = KAM_df["NMV target B2C"].sum()
    nmv_per_b2b = 0 if sum_nmv_target_b2b == 0 else sum_nmv_actual_b2b/sum_nmv_target_b2b
    nmv_per_b2c = 0 if sum_nmv_target_b2c == 0 else sum_nmv_actual_b2c/sum_nmv_target_b2c
    
    sum_gp_actual_b2b = KAM_df["GP actual B2B"].sum()
    sum_gp_actual_b2c = KAM_df["GP actual B2C"].sum()
    sum_gp_target_b2b = KAM_df["GP target B2B"].sum()
    sum_gp_target_b2c = KAM_df["GP target B2C"].sum()
    gp_per_b2b = 0 if sum_gp_target_b2b == 0 else sum_gp_actual_b2b/sum_gp_target_b2b
    gp_per_b2c = 0 if sum_gp_target_b2c == 0 else sum_gp_actual_b2c/sum_gp_target_b2c
    
    sum_slob_before = cal_sum_slob(KAM_df, "SLOB Before")
    sum_slob_after = cal_sum_slob(KAM_df, "SLOB After")
    sum_slob = cal_sum_slob(KAM_df, "SLOB")
    slop_per = 0 if sum_slob_before == 0 else sum_slob/sum_slob_before

    KAM_df.loc[len(KAM_df.index)] = [KAM_name, no_group_brand, no_store, "KAM", sum_nmv_actual_b2b, sum_nmv_target_b2b,
                                     nmv_per_b2b, sum_nmv_actual_b2c, sum_nmv_target_b2c, nmv_per_b2c,
                                     sum_gp_actual_b2b, sum_gp_target_b2b, gp_per_b2b,
                                     sum_gp_actual_b2c, sum_gp_target_b2c, gp_per_b2c,
                                     slop_per, sum_slob_before, sum_slob_after, sum_slob]
    
    # Change to percentage
    conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance"]
    for col_name in conv_to_per:
        KAM_df[col_name] = int_to_per(KAM_df, col_name)
        
    # We dont send the sum_slob_before, sum_slob_after, sum_slob
    KAM_df = KAM_df.drop(["SLOB Before", "SLOB After", "SLOB"], axis = 1)

    return KAM_df

In [59]:
KAMs = df["KAM/KAE (Full name)"].unique()
len(KAMs)

55

## Export KAM excel

In [31]:
if not os.path.exists('result_ceo'):
    os.mkdir('result_ceo')

df_final = take_detail_KAM(KAMs[0])
for i in range(1,len(KAMs)):
    df_current = take_detail_KAM(KAMs[i])
    df_final = pd.concat([df_final, df_current])
df_final = df_final.reset_index(drop=True)
df_final = df_final.style.apply(highlight_last_row, axis=1)
df_final.to_excel("result_ceo/KAM.xlsx", index = False, sheet_name = "KAM")

## Export GBM excel

In [60]:
# Write the func for creating df for each GBM
def take_GBM_df(KAM_name: str = None, GBM_name: str = None, CD_name: str = None):
    KAM_df = df
    if KAM_name != None:
        KAM_df = df[df["KAM/KAE (Full name)"] == KAM_name]
    if GBM_name != None:
        KAM_df = KAM_df[KAM_df["GBM/Team lead (Full name)"] == GBM_name]
    if CD_name != None:
        KAM_df = KAM_df[KAM_df["CD (Full name)"] == CD_name]
    cols = ["GBM/Team lead (Full name)", "Group brand", "Store", "B2B or B2C", "NMV actual",
            'Target NMV after', "% NMV",'GP actual (Simulated)', 'Target GP after (Simulated)',
            "% GP", "SLOB performance", 'SLOB Before', 'SLOB After', 'SLOB']
    KAM_df = KAM_df[cols]
    return KAM_df

In [69]:
# # Func for highlight_last_row
# def highlight_GBM(df):
#     color = 'white'
#     if df.values[3] == 'GBM':
#         color = 'yellow'
#     return ['background-color: %s' % color]*len(df.values)

# def take_detail_GBM(KAM_name: str = None, GBM_name: str = None, CD_name:str = None):
#     KAM_df = take_GBM_df(KAM_name, GBM_name, CD_name)
#     # extract b2b and b2c
#     KAM_df = extract_b_or_c(KAM_df)
    
#     # Add Total row
#     no_group_brand = len(KAM_df["Group brand"].unique())
#     no_store = len(KAM_df["Store"].unique())
    
#     sum_nmv_actual_b2b = KAM_df["NMV actual B2B"].sum()
#     sum_nmv_actual_b2c = KAM_df["NMV actual B2C"].sum()
#     sum_nmv_target_b2b = KAM_df["NMV target B2B"].sum()
#     sum_nmv_target_b2c = KAM_df["NMV target B2C"].sum()
#     nmv_per_b2b = 0 if sum_nmv_target_b2b == 0 else sum_nmv_actual_b2b/sum_nmv_target_b2b
#     nmv_per_b2c = 0 if sum_nmv_target_b2c == 0 else sum_nmv_actual_b2c/sum_nmv_target_b2c
    
#     sum_gp_actual_b2b = KAM_df["GP actual B2B"].sum()
#     sum_gp_actual_b2c = KAM_df["GP actual B2C"].sum()
#     sum_gp_target_b2b = KAM_df["GP target B2B"].sum()
#     sum_gp_target_b2c = KAM_df["GP target B2C"].sum()
#     gp_per_b2b = 0 if sum_gp_target_b2b == 0 else sum_gp_actual_b2b/sum_gp_target_b2b
#     gp_per_b2c = 0 if sum_gp_target_b2c == 0 else sum_gp_actual_b2c/sum_gp_target_b2c
    
#     sum_slob_before = cal_sum_slob(KAM_df, "SLOB Before")
#     sum_slob_after = cal_sum_slob(KAM_df, "SLOB After")
#     sum_slob = cal_sum_slob(KAM_df, "SLOB")
#     slop_per = 0 if sum_slob_before == 0 else sum_slob/sum_slob_before

#     print(len(KAM_df.index))
    
#     KAM_df.loc[len(KAM_df.index)] = [GBM_name, no_group_brand, no_store, "GBM", sum_nmv_actual_b2b, sum_nmv_target_b2b,
#                                      nmv_per_b2b, sum_nmv_actual_b2c, sum_nmv_target_b2c, nmv_per_b2c,
#                                      sum_gp_actual_b2b, sum_gp_target_b2b, gp_per_b2b,
#                                      sum_gp_actual_b2c, sum_gp_target_b2c, gp_per_b2c,
#                                      slop_per, sum_slob_before, sum_slob_after, sum_slob]
    
#     # Change to percentage
#     conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance"]
#     for col_name in conv_to_per:
#         KAM_df[col_name] = int_to_per(KAM_df, col_name)
        
#     # We dont send the sum_slob_before, sum_slob_after, sum_slob
#     KAM_df = KAM_df.drop(["SLOB Before", "SLOB After", "SLOB"], axis = 1)

#     return KAM_df


In [84]:
# Func for highlight_last_row
def highlight_GBM(df):
    color = 'white'
    if df.values[3] == 'GBM':
        color = 'yellow'
    return ['background-color: %s' % color]*len(df.values)

def take_detail_GBM(KAM_name: str = None, GBM_name: str = None, CD_name:str = None):
    KAM_df = take_GBM_df(KAM_name, GBM_name, CD_name)
    # extract b2b and b2c
    KAM_df = extract_b_or_c(KAM_df)
    
    # Add Total row
    no_group_brand = len(KAM_df["Group brand"].unique())
    no_store = len(KAM_df["Store"].unique())
    
    sum_nmv_actual_b2b = KAM_df["NMV actual B2B"].sum()
    sum_nmv_actual_b2c = KAM_df["NMV actual B2C"].sum()
    sum_nmv_target_b2b = KAM_df["NMV target B2B"].sum()
    sum_nmv_target_b2c = KAM_df["NMV target B2C"].sum()
    nmv_per_b2b = 0 if sum_nmv_target_b2b == 0 else sum_nmv_actual_b2b/sum_nmv_target_b2b
    nmv_per_b2c = 0 if sum_nmv_target_b2c == 0 else sum_nmv_actual_b2c/sum_nmv_target_b2c
    
    sum_gp_actual_b2b = KAM_df["GP actual B2B"].sum()
    sum_gp_actual_b2c = KAM_df["GP actual B2C"].sum()
    sum_gp_target_b2b = KAM_df["GP target B2B"].sum()
    sum_gp_target_b2c = KAM_df["GP target B2C"].sum()
    gp_per_b2b = 0 if sum_gp_target_b2b == 0 else sum_gp_actual_b2b/sum_gp_target_b2b
    gp_per_b2c = 0 if sum_gp_target_b2c == 0 else sum_gp_actual_b2c/sum_gp_target_b2c
    
    sum_slob_before = cal_sum_slob(KAM_df, "SLOB Before")
    sum_slob_after = cal_sum_slob(KAM_df, "SLOB After")
    sum_slob = cal_sum_slob(KAM_df, "SLOB")
    slop_per = 0 if sum_slob_before == 0 else sum_slob/sum_slob_before

    
    KAM_df.loc[-1] = [GBM_name, no_group_brand, no_store, "GBM", sum_nmv_actual_b2b, sum_nmv_target_b2b,
                                     nmv_per_b2b, sum_nmv_actual_b2c, sum_nmv_target_b2c, nmv_per_b2c,
                                     sum_gp_actual_b2b, sum_gp_target_b2b, gp_per_b2b,
                                     sum_gp_actual_b2c, sum_gp_target_b2c, gp_per_b2c,
                                     slop_per, sum_slob_before, sum_slob_after, sum_slob]
    
    # Change to percentage
    conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance"]
    for col_name in conv_to_per:
        KAM_df[col_name] = int_to_per(KAM_df, col_name)
        
    # We dont send the sum_slob_before, sum_slob_after, sum_slob
    KAM_df = KAM_df.drop(["SLOB Before", "SLOB After", "SLOB"], axis = 1)

    return KAM_df

In [85]:
df_final = take_detail_GBM(GBM_name = GBMs[0])
for i in range(1,len(GBMs)):
    df_current = take_detail_GBM(GBM_name = GBMs[i])
    df_final = pd.concat([df_final, df_current])
df_final = df_final.reset_index(drop=True)
df_final = df_final.style.apply(highlight_GBM, axis=1)
df_final.to_excel("result_ceo/GBM.xlsx", index = False, sheet_name = "GBM")

## Export CD excel

In [35]:
# Write the func for creating df for each CD
def take_CD_df(KAM_name: str = None, GBM_name: str = None, CD_name: str = None):
    KAM_df = df
    if KAM_name != None:
        KAM_df = df[df["KAM/KAE (Full name)"] == KAM_name]
    if GBM_name != None:
        KAM_df = KAM_df[KAM_df["GBM/Team lead (Full name)"] == GBM_name]
    if CD_name != None:
        KAM_df = KAM_df[KAM_df["CD (Full name)"] == CD_name]
    cols = ["CD (Full name)", "Group brand", "Store", "B2B or B2C", "NMV actual",
            'Target NMV after', "% NMV",'GP actual (Simulated)', 'Target GP after (Simulated)',
            "% GP", "SLOB performance", 'SLOB Before', 'SLOB After', 'SLOB']
    KAM_df = KAM_df[cols]
    return KAM_df

In [36]:
# Func for highlight_last_row
def highlight_CD(df):
    color = 'white'
    if df.values[3] == 'CD':
        color = 'yellow'
    return ['background-color: %s' % color]*len(df.values)

def take_detail_CD(KAM_name: str = None, GBM_name: str = None, CD_name:str = None):
    KAM_df = take_CD_df(KAM_name, GBM_name, CD_name)
    # extract b2b and b2c
    KAM_df = extract_b_or_c(KAM_df)
    
    # Add Total row
    no_group_brand = len(KAM_df["Group brand"].unique())
    no_store = len(KAM_df["Store"].unique())
    
    sum_nmv_actual_b2b = KAM_df["NMV actual B2B"].sum()
    sum_nmv_actual_b2c = KAM_df["NMV actual B2C"].sum()
    sum_nmv_target_b2b = KAM_df["NMV target B2B"].sum()
    sum_nmv_target_b2c = KAM_df["NMV target B2C"].sum()
    nmv_per_b2b = 0 if sum_nmv_target_b2b == 0 else sum_nmv_actual_b2b/sum_nmv_target_b2b
    nmv_per_b2c = 0 if sum_nmv_target_b2c == 0 else sum_nmv_actual_b2c/sum_nmv_target_b2c
    
    sum_gp_actual_b2b = KAM_df["GP actual B2B"].sum()
    sum_gp_actual_b2c = KAM_df["GP actual B2C"].sum()
    sum_gp_target_b2b = KAM_df["GP target B2B"].sum()
    sum_gp_target_b2c = KAM_df["GP target B2C"].sum()
    gp_per_b2b = 0 if sum_gp_target_b2b == 0 else sum_gp_actual_b2b/sum_gp_target_b2b
    gp_per_b2c = 0 if sum_gp_target_b2c == 0 else sum_gp_actual_b2c/sum_gp_target_b2c
    
    sum_slob_before = cal_sum_slob(KAM_df, "SLOB Before")
    sum_slob_after = cal_sum_slob(KAM_df, "SLOB After")
    sum_slob = cal_sum_slob(KAM_df, "SLOB")
    slop_per = 0 if sum_slob_before == 0 else sum_slob/sum_slob_before
    KAM_df.loc[len(KAM_df.index)] = [CD_name, no_group_brand, no_store, "CD", sum_nmv_actual_b2b, sum_nmv_target_b2b,
                                     nmv_per_b2b, sum_nmv_actual_b2c, sum_nmv_target_b2c, nmv_per_b2c,
                                     sum_gp_actual_b2b, sum_gp_target_b2b, gp_per_b2b,
                                     sum_gp_actual_b2c, sum_gp_target_b2c, gp_per_b2c,
                                     slop_per, sum_slob_before, sum_slob_after, sum_slob]
    
    # Change to percentage
    conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance"]
    for col_name in conv_to_per:
        KAM_df[col_name] = int_to_per(KAM_df, col_name)
        
    # We dont send the sum_slob_before, sum_slob_after, sum_slob
    KAM_df = KAM_df.drop(["SLOB Before", "SLOB After", "SLOB"], axis = 1)

    return KAM_df


In [37]:
df_final = take_detail_CD(CD_name = CDs[0])
for i in range(1,len(CDs)):
    df_current = take_detail_CD(CD_name = CDs[i])
    df_final = pd.concat([df_final, df_current])
df_final = df_final.reset_index(drop=True)
df_final = df_final.style.apply(highlight_CD, axis=1)
df_final.to_excel("result_ceo/CD.xlsx", index = False, sheet_name = "CD")

# Sheet General

## Define the threshold and incentive

In [38]:
# Define the threshold of calculating the incentive
NMV_threshold = [0.85, 1, 1.25, 1.5]
GP_threshold = [0.85, 1, 1.25, 1.5]
SLOB_threshold = [-0.15]

# Define the incentive based on threshold
NMV_incentive_values = [0.02, 0.08, 0.12, 0.2]
NMV_incentive_B2B_values = [0.004, 0.016, 0.024, 0.04]
NMV_incentive_B2C_values = [0.016, 0.064, 0.096, 0.16]

GP_incentive_values = [0.03, 0.12, 0.18, 0.3]
GP_incentive_B2B_values = [0.006, 0.024, 0.036, 0.06]
GP_incentive_B2C_values = [0.024, 0.096, 0.144, 0.24]

SLOB_incentive_values = [0.05]

## Function Calculate the incentive

In [39]:
def cal_incentive(role: str):    
    # Take all name of role
    if role == "KAM":
        role_col = "KAM/KAE (Full name)"
        NMV_target_col = "Target NMV after"
        GP_target_col = "Target GP after (Simulated)"
        
    elif role == "GBM":
        role_col = "GBM/Team lead (Full name)"
        NMV_target_col = "Target NMV after"
        GP_target_col = "Target GP after (Simulated)"
        
    elif role == "CD":
        role_col = "CD (Full name)"
        NMV_target_col = "Target NMV BP"
        GP_target_col = "Target GP BP (simulated)"
    
    people = list(df[role_col].unique())
   
    # Calculate the incentive of each person
    people_incentive = []
    
    people_total_NMV_actual_B2B = []
    people_total_NMV_target_B2B = []
    people_percentage_NMV_B2B = []
    
    people_total_GP_actual_B2B = []
    people_total_GP_target_B2B = []
    people_percentage_GP_B2B = []
    
    people_total_NMV_actual_B2C = []
    people_total_NMV_target_B2C = []
    people_percentage_NMV_B2C = []
    
    people_total_GP_actual_B2C = []
    people_total_GP_target_B2C = []
    people_percentage_GP_B2C = []
    
    people_total_SLOB_performance = []
    
    people_incentive_NMV = []
    people_incentive_GP = []
    people_incentive_SLOB = []
    
    
    for person in people:
        person_df = df.loc[df[role_col] == person]

        # only B2C
        if "B2B" not in person_df["B2B or B2C"].unique():
            # Cal incentive of NMV
            person_NMV_actual = person_df["NMV actual"].sum()
            person_NMV_target = person_df[NMV_target_col].sum()
            person_percentage_NMV = person_NMV_actual/person_NMV_target if person_NMV_target != 0 else 0
            if person_percentage_NMV >= NMV_threshold[0] and person_percentage_NMV < NMV_threshold[1]:
                person_incentive_NMV = NMV_incentive_values[0]
            elif person_percentage_NMV >= NMV_threshold[1] and person_percentage_NMV < NMV_threshold[2]:
                person_incentive_NMV = NMV_incentive_values[1]
            elif person_percentage_NMV >= NMV_threshold[2] and person_percentage_NMV < NMV_threshold[3]:
                person_incentive_NMV = NMV_incentive_values[2]
            elif person_percentage_NMV >= NMV_threshold[3]:
                person_incentive_NMV = NMV_incentive_values[3]
            else:
                person_incentive_NMV = 0

            # Cal incentive of GP
            person_GP_actual = person_df["GP actual (Simulated)"].sum()
            person_GP_target = person_df[GP_target_col].sum()
            person_percentage_GP = person_GP_actual/person_GP_target if person_GP_target != 0 else 0

            if person_percentage_GP >= GP_threshold[0] and person_percentage_GP < GP_threshold[1]:
                person_incentive_GP = GP_incentive_values[0]
            elif person_percentage_GP >= GP_threshold[1] and person_percentage_GP < GP_threshold[2]:
                person_incentive_GP = GP_incentive_values[1]
            elif person_percentage_GP >= GP_threshold[2] and person_percentage_GP < GP_threshold[3]:
                person_incentive_GP = GP_incentive_values[2]
            elif person_percentage_GP >= GP_threshold[3]:
                person_incentive_GP = GP_incentive_values[3]
            else:
                person_incentive_GP = 0
                
            # Add to list
            people_total_NMV_actual_B2C.append(person_NMV_actual)
            people_total_NMV_target_B2C.append(person_NMV_target)
            people_percentage_NMV_B2C.append(person_percentage_NMV)
            
            people_total_GP_actual_B2C.append(person_GP_actual)
            people_total_GP_target_B2C.append(person_GP_target)
            people_percentage_GP_B2C.append(person_percentage_GP)
            
            people_total_NMV_actual_B2B.append("")
            people_total_NMV_target_B2B.append("")
            people_percentage_NMV_B2B.append("")
            
            people_total_GP_actual_B2B.append("")
            people_total_GP_target_B2B.append("")
            people_percentage_GP_B2B.append("")

        # B2B mixed with B2C
        else:
            person_df_B2B = person_df.loc[df["B2B or B2C"] == "B2B"]
            person_df_B2C = person_df.loc[df["B2B or B2C"] == "B2C"]

            # Cal incentive of NMV
            # B2B
            person_NMV_actual_B2B = person_df_B2B["NMV actual"].sum()
            person_NMV_target_B2B = person_df_B2B[NMV_target_col].sum()
            
            #person_percentage_NMV_B2B = person_NMV_actual_B2B/person_NMV_target_B2B if person_NMV_target_B2B != 0 else 0
            if person_NMV_target_B2B == 0:
                if person_NMV_actual_B2B != 0:
                    person_percentage_NMV_B2B = 1
                else:
                    person_percentage_NMV_B2B = 0
            else:
                person_percentage_NMV_B2B = person_NMV_actual_B2B/person_NMV_target_B2B
            #######
            if person_percentage_NMV_B2B >= NMV_threshold[0] and person_percentage_NMV_B2B < NMV_threshold[1]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[0]
            elif person_percentage_NMV_B2B >= NMV_threshold[1] and person_percentage_NMV_B2B < NMV_threshold[2]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[1]
            elif person_percentage_NMV_B2B >= NMV_threshold[2] and person_percentage_NMV_B2B < NMV_threshold[3]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[2]
            elif person_percentage_NMV_B2B >= NMV_threshold[3]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[3]
            else:
                person_incentive_NMV_B2B = 0
        
            # B2C
            person_NMV_actual_B2C = person_df_B2C["NMV actual"].sum()
            person_NMV_target_B2C = person_df_B2C[NMV_target_col].sum()
            person_percentage_NMV_B2C = person_NMV_actual_B2C/person_NMV_target_B2C if person_NMV_target_B2C != 0 else 0
            if person_percentage_NMV_B2C >= NMV_threshold[0] and person_percentage_NMV_B2C < NMV_threshold[1]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[0]
            elif person_percentage_NMV_B2C >= NMV_threshold[1] and person_percentage_NMV_B2C < NMV_threshold[2]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[1]
            elif person_percentage_NMV_B2C >= NMV_threshold[2] and person_percentage_NMV_B2C < NMV_threshold[3]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[2]
            elif person_percentage_NMV_B2C >= NMV_threshold[3]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[3]
            else:
                person_incentive_NMV_B2C = 0

            person_incentive_NMV = person_incentive_NMV_B2B + person_incentive_NMV_B2C

            # Cal incentive of GP
            # B2B
            person_GP_actual_B2B = person_df_B2B["GP actual (Simulated)"].sum()
            person_GP_target_B2B = person_df_B2B[GP_target_col].sum()
#             person_percentage_GP_B2B = person_GP_actual_B2B/person_GP_target_B2B if person_GP_target_B2B != 0 else 0
            if person_GP_target_B2B == 0:
                if person_GP_actual_B2B != 0:
                    person_percentage_GP_B2B = 1
                else:
                    person_percentage_GP_B2B = 0
            else:
                person_percentage_GP_B2B = person_GP_actual_B2B/person_GP_target_B2B
            #######


            if person_percentage_GP_B2B >= GP_threshold[0] and person_percentage_GP_B2B < GP_threshold[1]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[0]
            elif person_percentage_GP_B2B >= GP_threshold[1] and person_percentage_GP_B2B < GP_threshold[2]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[1]
            elif person_percentage_GP_B2B >= GP_threshold[2] and person_percentage_GP_B2B < GP_threshold[3]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[2]
            elif person_percentage_GP_B2B >= GP_threshold[3]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[3]
            else:
                person_incentive_GP_B2B = 0

            # B2C
            person_GP_actual_B2C = person_df_B2C["GP actual (Simulated)"].sum()
            person_GP_target_B2C = person_df_B2C[GP_target_col].sum()
            person_percentage_GP_B2C = person_GP_actual_B2C/person_GP_target_B2C if person_GP_target_B2C != 0 else 0

            if person_percentage_GP_B2C >= GP_threshold[0] and person_percentage_GP_B2C < GP_threshold[1]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[0]
            elif person_percentage_GP_B2C >= GP_threshold[1] and person_percentage_GP_B2C < GP_threshold[2]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[1]
            elif person_percentage_GP_B2C >= GP_threshold[2] and person_percentage_GP_B2C < GP_threshold[3]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[2]
            elif person_percentage_GP_B2C >= GP_threshold[3]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[3]
            else:
                person_incentive_GP_B2C = 0

            person_incentive_GP = person_incentive_GP_B2B + person_incentive_GP_B2C
            
            # Add to list
            people_total_NMV_actual_B2C.append(person_NMV_actual_B2C)
            people_total_NMV_target_B2C.append(person_NMV_target_B2C)
            
            people_total_GP_actual_B2C.append(person_GP_actual_B2C)
            people_total_GP_target_B2C.append(person_GP_target_B2C)
            
            people_total_NMV_actual_B2B.append(person_NMV_actual_B2B)
            people_total_NMV_target_B2B.append(person_NMV_target_B2B)
            
            people_total_GP_actual_B2B.append(person_GP_actual_B2B)
            people_total_GP_target_B2B.append(person_GP_target_B2B)
            
            people_percentage_NMV_B2C.append(person_percentage_NMV_B2C)
            people_percentage_GP_B2C.append(person_percentage_GP_B2C)
            people_percentage_NMV_B2B.append(person_percentage_NMV_B2B)
            people_percentage_GP_B2B.append(person_percentage_GP_B2B)
            
        # Cal incentive of SLOB
        if person_df["SLOB Before"].sum() == 0:
            person_incentive_SLOB = 0
            person_percentage_SLOB = 0
        else:
            person_groupbrand = person_df["Group brand"].to_list()
            person_SLOB_Minus = person_df["SLOB"].to_list()
            person_SLOB_Before = person_df["SLOB Before"].to_list()
            k = ""
            sum_Minus = 0
            sum_Before = 0
            for i in range(len(person_groupbrand)):
                if person_groupbrand[i] == k:
                    continue
                else:
                    sum_Minus += person_SLOB_Minus[i]
                    sum_Before += person_SLOB_Before[i]
                    k = person_groupbrand[i]  
            person_percentage_SLOB = sum_Minus / sum_Before
            if person_percentage_SLOB <= SLOB_threshold[0]:
                person_incentive_SLOB = SLOB_incentive_values[0]
            else:
                    person_incentive_SLOB = 0

        # Sum all incentive of KAM
        person_incentive = person_incentive_NMV + person_incentive_GP + person_incentive_SLOB

        # Add to list
        people_incentive_NMV.append(person_incentive_NMV)
        people_incentive_GP.append(person_incentive_GP)
        people_incentive_SLOB.append(person_incentive_SLOB)
        
        people_incentive.append(person_incentive)
        people_total_SLOB_performance.append(person_percentage_SLOB)

    return people, people_total_NMV_actual_B2B, people_total_NMV_target_B2B, people_percentage_NMV_B2B,people_total_GP_actual_B2B, people_total_GP_target_B2B, people_percentage_GP_B2B, people_total_NMV_actual_B2C, people_total_NMV_target_B2C, people_percentage_NMV_B2C, people_total_GP_actual_B2C, people_total_GP_target_B2C, people_percentage_GP_B2C, people_total_SLOB_performance,people_incentive_NMV, people_incentive_GP, people_incentive_SLOB, people_incentive,

### Create the KAM_df

In [40]:
KAMs, KAM_total_NMV_actual_B2B, KAM_total_NMV_target_B2B, KAM_percentage_NMV_B2B, KAM_total_GP_actual_B2B, KAM_total_GP_target_B2B, KAM_percentage_GP_B2B,KAM_total_NMV_actual_B2C, KAM_total_NMV_target_B2C, KAM_percentage_NMV_B2C, KAM_total_GP_actual_B2C, KAM_total_GP_target_B2C, KAM_percentage_GP_B2C, KAM_total_SLOB_performance, KAM_incentive_NMV, KAM_incentive_GP, KAM_incentive_SLOB, KAM_incentive  = cal_incentive("KAM")
gbm_list, GBM_total_NMV_actual_B2B, GBM_total_NMV_target_B2B, GBM_percentage_NMV_B2B, GBM_total_GP_actual_B2B, GBM_total_GP_target_B2B, GBM_percentage_GP_B2B,GBM_total_NMV_actual_B2C, GBM_total_NMV_target_B2C, GBM_percentage_NMV_B2C, GBM_total_GP_actual_B2C, GBM_total_GP_target_B2C, GBM_percentage_GP_B2C, GBM_total_SLOB_performance, GBM_incentive_NMV, GBM_incentive_GP, GBM_incentive_SLOB, GBM_incentive  = cal_incentive("GBM")
cd_list, CD_total_NMV_actual_B2B, CD_total_NMV_target_B2B, CD_percentage_NMV_B2B, CD_total_GP_actual_B2B, CD_total_GP_target_B2B, CD_percentage_GP_B2B,CD_total_NMV_actual_B2C, CD_total_NMV_target_B2C, CD_percentage_NMV_B2C, CD_total_GP_actual_B2C, CD_total_GP_target_B2C, CD_percentage_GP_B2C, CD_total_SLOB_performance, CD_incentive_NMV, CD_incentive_GP, CD_incentive_SLOB, CD_incentive  = cal_incentive("CD")

excel_df_cols = ["Name", "NMV actual B2B", "NMV target B2B", "% NMV B2B","NMV actual B2C", "NMV target B2C","% NMV B2C", "GP actual B2B", "GP target B2B", "% GP B2B","GP actual B2C", "GP target B2C", "% GP B2C","SLOB performance","NMV Incentive", "GP Incentive", "SLOB Incentive", "Incentive"]

In [41]:
KAMs_df = pd.DataFrame(list(zip(KAMs, KAM_total_NMV_actual_B2B, KAM_total_NMV_target_B2B, KAM_percentage_NMV_B2B, KAM_total_NMV_actual_B2C, KAM_total_NMV_target_B2C, KAM_percentage_NMV_B2C, KAM_total_GP_actual_B2B, KAM_total_GP_target_B2B, KAM_percentage_GP_B2B, KAM_total_GP_actual_B2C, KAM_total_GP_target_B2C, KAM_percentage_GP_B2C, KAM_total_SLOB_performance, KAM_incentive_NMV, KAM_incentive_GP, KAM_incentive_SLOB, KAM_incentive)),
                  columns = excel_df_cols)
conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance", "Incentive", "NMV Incentive", "GP Incentive", "SLOB Incentive"]
for col_name in conv_to_per:
    KAMs_df[col_name] = int_to_per(KAMs_df, col_name)

In [42]:
KAMs_df = KAMs_df.reset_index(drop=True)
KAMs_df = KAMs_df.style.apply(highlight_last_row, axis=1)
KAMs_df.to_excel("result_ceo/KAM_incentive.xlsx", index = False, sheet_name = "KAM Incentive")


### Create the GBMs_df

In [43]:

GBMs_df = pd.DataFrame(list(zip(gbm_list, GBM_total_NMV_actual_B2B, GBM_total_NMV_target_B2B, GBM_percentage_NMV_B2B, GBM_total_NMV_actual_B2C, GBM_total_NMV_target_B2C, GBM_percentage_NMV_B2C, GBM_total_GP_actual_B2B, GBM_total_GP_target_B2B, GBM_percentage_GP_B2B, GBM_total_GP_actual_B2C, GBM_total_GP_target_B2C, GBM_percentage_GP_B2C, GBM_total_SLOB_performance, GBM_incentive_NMV, GBM_incentive_GP, GBM_incentive_SLOB, GBM_incentive)),
                  columns = excel_df_cols)
for col_name in conv_to_per:
    GBMs_df[col_name] = int_to_per(GBMs_df, col_name)
    
GBMs_df = GBMs_df.reset_index(drop=True)
GBMs_df = GBMs_df.style.apply(highlight_last_row, axis=1)
GBMs_df.to_excel("result_ceo/GBM_incentive.xlsx", index = False, sheet_name = "GBM Incentive")

### Create the CDs_df

In [44]:

CDs_df = pd.DataFrame(list(zip(cd_list, CD_total_NMV_actual_B2B, CD_total_NMV_target_B2B, CD_percentage_NMV_B2B, CD_total_NMV_actual_B2C, CD_total_NMV_target_B2C, CD_percentage_NMV_B2C, CD_total_GP_actual_B2B, CD_total_GP_target_B2B, CD_percentage_GP_B2B, CD_total_GP_actual_B2C, CD_total_GP_target_B2C, CD_percentage_GP_B2C, CD_total_SLOB_performance, CD_incentive_NMV, CD_incentive_GP, CD_incentive_SLOB, CD_incentive)),
                  columns = excel_df_cols)

for col_name in conv_to_per:
    CDs_df[col_name] = int_to_per(CDs_df, col_name)
    
CDs_df = CDs_df.reset_index(drop=True)
CDs_df = CDs_df.style.apply(highlight_last_row, axis=1)
CDs_df.to_excel("result_ceo/CD_incentive.xlsx", index = False, sheet_name = "CD Incentive")

In [47]:
GBMs

array(['Nguyễn Thị Hồng Nhung', 'Đặng Thị Huyền', 'Nguyễn Châu Hoàng Yến',
       'Nguyễn Thị Bảo Trân', 'Bùi Duy Anh', 'Trần Tiến Đức',
       'Vũ Thị Phương Thảo', 'Trần Trọng Hiếu', 'Lê Ngọc Hân',
       'Nguyễn An Minh Thái', 'Lê Chiến Thắng', 'Nguyễn Văn Lãm'],
      dtype=object)

In [79]:
take_detail_GBM('Bùi Duy Anh')

0


9,GBM/Team lead (Full name),Group brand,Store,B2B or B2C,NMV actual B2B,NMV target B2B,% NMV B2B,NMV actual B2C,NMV target B2C,% NMV B2C,GP actual B2B,GP target B2B,% GP B2B,GP actual B2C,GP target B2C,% GP B2C,SLOB performance
-1,None,0,0,GBM,0.0,0.0,0%,0.0,0.0,0%,0.0,0.0,0%,0.0,0.0,0%,0%


In [81]:
take_detail_GBM(GBM_name= GBMs[2]).shape


18


(19, 17)